## SYS_5581_Project

### Setup

Before we can perform analysis and scenarios on the LMP data we must first load the data into useful dataframes. Here we load our csv files into data frames for each month and a combined dataframe for the whole year.

In [12]:
import os
import pandas as pd

data_dir = "./data"

# list of dataframes from each month of the year
monthly_dfs = []

for folder_name in os.listdir(data_dir):
    folder = data_dir + "/" + folder_name
    
    # dfs for each day of the month
    days_dfs = []
    for file in os.listdir(folder):
        file_path = folder + "/" + file
        days_dfs.append(pd.read_csv(file_path))
        
    # df for the entire month
    month_df = pd.concat(days_dfs, axis=0, ignore_index=True)
    monthly_dfs.append(month_df)
    
# dataframe for full year worth of data
year_df = pd.concat(monthly_dfs, axis=0, ignore_index=True)

print(year_df)
        
        


                  Time Stamp    Name   PTID  LBMP ($/MWHr)  \
0        03/02/2019 00:05:00  CAPITL  61757          33.95   
1        03/02/2019 00:05:00  CENTRL  61754          31.99   
2        03/02/2019 00:05:00  DUNWOD  61760          33.88   
3        03/02/2019 00:05:00  GENESE  61753          31.17   
4        03/02/2019 00:05:00     H Q  61844          30.63   
...                      ...     ...    ...            ...   
1602790  06/26/2019 00:00:00   NORTH  61755         -13.20   
1602791  06/26/2019 00:00:00     NPX  61845          19.80   
1602792  06/26/2019 00:00:00     O H  61846          17.25   
1602793  06/26/2019 00:00:00     PJM  61847          19.32   
1602794  06/26/2019 00:00:00    WEST  61752          17.54   

         Marginal Cost Losses ($/MWHr)  Marginal Cost Congestion ($/MWHr)  
0                                 2.37                               0.00  
1                                 0.41                               0.00  
2                          

### Historical Price Analysis

In this section we will analyze the historic LMP data we retrieved. We will look at the change in price over time in different regions of New York.

In [50]:
import matplotlib.pyplot as plt

genese = year_df[year_df["Name"] == "GENESE"]
dunwood = year_df[year_df["Name"] == "DUNWOD"]
central = year_df[year_df["Name"] == "CENTRL"]



#plt.hist(genese["LBMP ($/MWHr)"], 12, range=[1:12], density=True, histtype='bar', color='blue', alpha=0.75)

plt.plot(genese["LBMP ($/MWHr)"])
     
# plt.xlabel('Month') 
# plt.ylabel('LBMP ($/MWHr)') 
# plt.title("Genese LBMP for 2019")


# plt.show()


         


#

# plt.show


Time Stamp


3          31.17
18         31.06
33         31.17
48         31.17
63         31.10
           ...  
1602723    23.10
1602738    26.86
1602753    27.44
1602768    28.96
1602783    17.82
Name: LBMP ($/MWHr), Length: 106853, dtype: float64